In [11]:
%load_ext autoreload
%autoreload 2
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
df_raw = load_dataframe()

Loading dataframe from pickle file...


In [13]:
df_raw.head()

,text,label
0,US trade gap ballooned in October\n\nThe US tr...,business
1,Japanese mogul arrested for fraud\n\nOne of Ja...,business
2,Jarvis sells Tube stake to Spain\n\nShares in ...,business
3,Small firms 'hit by rising costs'\n\nRising fu...,business
4,Oil prices fall back from highs\n\nOil prices ...,business


In [4]:
PREPROCESSING_PIPELINE = [
                        lower,
                        remove_emails,
                        replace_special_characters,
                        replace_br,
                        filter_out_uncommon_symbols,
                        remove_stopwords,
                        strip_text
                    ]

In [5]:
print('Pre-processing text...')
 
print()
print(f'[Debug] Before:\n{df_raw.text.values[0]}')
print()
 
# Replace each sentence with its pre-processed version
df = df_raw.copy()
df['text'] = df_raw['text'].apply(lambda txt: text_prepare(txt, PREPROCESSING_PIPELINE))
 
print(f'[Debug] After:\n{df.text.values[0]}')
print()
 
print("Pre-processing completed!")

Pre-processing text...

[Debug] Before:
US trade gap ballooned in October

The US trade deficit widened by more than expected in October, hitting record levels after higher oil prices raised import costs, figures have shown

The trade shortfall was $55.5bn (£29bn), up 9% from September, the Commerce Department said. That pushed the 10 month deficit to $500.5bn. Imports rose by 3.4%, while exports increased by only 0.6%. A weaker dollar also increased the cost of imports, though this should help drive export demand in coming months. "Things are getting worse, but that's to be expected," said David Wyss of Standard & Poor's in New York. "The first thing is that when the dollar goes down, it increases the price of imports. "We are seeing improved export orders. Things seem to be going in the right direction."

Despite this optimism, significant concerns remain as to how the US will fund its trade and budget deficits should they continue to widen. Another problem highlighted by analysts wa

In [6]:
path = 'vocab/'
if not os.path.exists(path):
    os.makedirs(path)

# Check if vocabulary has already been built
if os.path.exists(path + 'idx_to_word.json') and os.path.exists(path + 'word_to_idx.json') and os.path.exists(path + 'word_listing.json'):
    print('Loading existing vocabulary...')
    idx_to_word, word_to_idx, word_listing = load_vocab()
    print('Vocabulary loaded!')
else:
    print('Building vocabulary...')
    idx_to_word, word_to_idx, word_listing = build_vocabulary(df)
    print('Saving vocabulary...')
    save_vocab(idx_to_word, word_to_idx, word_listing)
    print('Vocabulary built!')

Building vocabulary...


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 14769.09it/s]

Saving vocabulary...
Vocabulary built!


In [7]:
evaluate_vocabulary(idx_to_word, word_to_idx, word_listing, df)

[Vocabulary Evaluation] Size checking...
[Vocabulary Evaluation] Content checking...


100%|██████████| 31579/31579 [00:00<00:00, 1279802.95it/s]


[Vocabulary Evaluation] Consistency checking...


100%|██████████| 1000/1000 [00:00<00:00, 4089.63it/s]


[Vocabulary Evaluation] Toy example checking...


100%|██████████| 2/2 [00:00<00:00, 1682.77it/s]


In [8]:
df_train, df_test = split_dataframe(df)

In [9]:
print(f'Train size: {len(df_train)}')
df_train.head()

Train size: 800


,text,label
0,graphic design graphic design craft creating v...,graphics
1,wada appeal ruling world antidoping agency wad...,sport
2,george michael perform bbc george michael perf...,entertainment
3,yelling takes cardiff hattrick european crossc...,sport
4,analysis shows billions dollars spent advertis...,food


In [10]:
print(f'Test size: {len(df_test)}')
df_test.head()

Test size: 200


,text,label
0,looking current sources lists home medical tes...,medical
1,hi sure post message please contact im way mar...,medical
2,1 2 pound flank steak lean 2 teaspoons finely ...,food
3,omega3 fatty acids hype plethora nutrition inf...,food
4,reply md many metabolic tests body structure t...,medical
